In [11]:
import scanpy as sc
import pandas as pd
import harmonypy as hm
import anndata
import matplotlib.pyplot as plt

In [12]:
# Load the h5ad file
data_path = "/rds/general/user/tf424/projects/single_cell_pd/live/amppd_brain/merged_data_amppd/quantification/amppd_merged_snRNAseq_annotated.h5ad"
adata = anndata.read(data_path,backed='r')

/rds/general/user/tf424/home/miniforge3/envs/new_sc_env/lib/python3.10/site-packages/anndata/__init__.py:42: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [13]:
# Extract brain_region info
adata.obs['brain_region'] = adata.obs['sample_id'].str.extract(r'-(GPI|PMC|PVC|DMNX|PFC)-')

In [14]:
# Load demographics CSV
demographics = pd.read_csv("/rds/general/user/tf424/home/clinical/Demographics.csv")

# Check
print(demographics.columns)

Index(['participant_id', 'GUID', 'visit_name', 'visit_month',
       'age_at_baseline', 'sex', 'ethnicity', 'race', 'education_level_years'],
      dtype='object')


In [15]:
# Set index to match
demographics = demographics.set_index("participant_id")

# Add gender
adata.obs["sex"] = adata.obs["participant_id"].map(demographics["sex"])

In [16]:
adata.obs['sex'] = adata.obs['sex'].astype('category')

In [17]:
print(adata.obs.head())

                                              sample_id   participant_id  \
barcodekey                                                                 
Set10_C1-AAACCCACATCACGGC   PM-UM_BEB19139-BLM0-GPI-RSN   PM-UM_BEB19139   
Set10_C1-AAACCCAGTAGCACAG  PM-UD_11485_001-BLM0-PMC-RSN  PM-UD_11485_001   
Set10_C1-AAACCCAGTATGTCCA   PM-UM_BEB19139-BLM0-GPI-RSN   PM-UM_BEB19139   
Set10_C1-AAACCCAGTCCAGAAG  PM-UD_11485_001-BLM0-PMC-RSN  PM-UD_11485_001   
Set10_C1-AAACCCATCCACGTAA  PM-UD_11485_001-BLM0-PMC-RSN  PM-UD_11485_001   

                          cell_type brain_region     sex  
barcodekey                                                
Set10_C1-AAACCCACATCACGGC     Oligo          GPI    Male  
Set10_C1-AAACCCAGTAGCACAG     Oligo          PMC  Female  
Set10_C1-AAACCCAGTATGTCCA     Oligo          GPI    Male  
Set10_C1-AAACCCAGTCCAGAAG     Oligo          PMC  Female  
Set10_C1-AAACCCATCCACGTAA     Astro          PMC  Female  


In [18]:
# Load the CSV files
file_1 = "/rds/general/user/tf424/projects/ppmi_verily/live/AMP-PD/participants/amp_pd_case_control.csv"
file_2 = "/rds/general/user/tf424/projects/ppmi_verily/live/AMP-PD/participants/amp_pd_participants.csv"
# Convert into df
df_1 = pd.read_csv(file_1)
df_3 = pd.read_csv(file_2)
# Merge
merged_data = pd.merge(df_1, df_3, on='participant_id', how='inner')

In [19]:
participants_data = merged_data
merged_obs = pd.merge(adata.obs, participants_data, on='participant_id', how='inner')
merged_obs = merged_obs.reset_index(drop=True)
adata.obs = merged_obs

# Check
print(adata.obs.head())

                      sample_id   participant_id cell_type brain_region  \
0   PM-UM_BEB19139-BLM0-GPI-RSN   PM-UM_BEB19139     Oligo          GPI   
1  PM-UD_11485_001-BLM0-PMC-RSN  PM-UD_11485_001     Oligo          PMC   
2   PM-UM_BEB19139-BLM0-GPI-RSN   PM-UM_BEB19139     Oligo          GPI   
3  PM-UD_11485_001-BLM0-PMC-RSN  PM-UD_11485_001     Oligo          PMC   
4  PM-UD_11485_001-BLM0-PMC-RSN  PM-UD_11485_001     Astro          PMC   

      sex diagnosis_at_baseline     diagnosis_latest  \
0    Male   Parkinson's Disease  Parkinson's Disease   
1  Female   Parkinson's Disease  Parkinson's Disease   
2    Male   Parkinson's Disease  Parkinson's Disease   
3  Female   Parkinson's Disease  Parkinson's Disease   
4  Female   Parkinson's Disease  Parkinson's Disease   

  case_control_other_at_baseline case_control_other_latest guid  \
0                           Case                      Case  NaN   
1                           Case                      Case  NaN   
2          

In [ ]:
# Access obs and count cells per participant
obs = adata.obs
cell_counts = obs['participant_id'].value_counts()
min_cells_required = 500
eligible_ids = cell_counts[cell_counts >= min_cells_required].index.tolist()

# Subset obs to only eligible participants
meta = obs[obs['participant_id'].isin(eligible_ids)][
    ['participant_id', 'sex', 'case_control_other_at_baseline']
].drop_duplicates()

# Sample 5 PD and 5 Control per gender
female_pd = meta[(meta['sex'] == 'Female') & 
                 (meta['case_control_other_at_baseline'] == 'Case')]['participant_id'].sample(5, random_state=1)

female_control = meta[(meta['sex'] == 'Female') & 
                      (meta['case_control_other_at_baseline'] == 'Control')]['participant_id'].sample(5, random_state=1)

male_pd = meta[(meta['sex'] == 'Male') & 
               (meta['case_control_other_at_baseline'] == 'Case')]['participant_id'].sample(5, random_state=1)

male_control = meta[(meta['sex'] == 'Male') & 
                    (meta['case_control_other_at_baseline'] == 'Control')]['participant_id'].sample(5, random_state=1)

selected_ids = pd.concat([female_pd, female_control, male_pd, male_control]).tolist()

In [ ]:
# Save list of selected cell barcodes
selected_obs = obs[obs['participant_id'].isin(selected_ids)]

#  Add checks here
meta_check = selected_obs[['participant_id', 'sex', 'case_control_other_at_baseline']].drop_duplicates()

# 1. Check participant balance
print(" Participant counts by sex and diagnosis")
print(meta_check.groupby(['sex', 'case_control_other_at_baseline']).size().unstack(fill_value=0))

# 2. Check cell counts per participant
cell_counts_check = selected_obs['participant_id'].value_counts().sort_values()
print("Cell counts per participant")
print(cell_counts_check)

if (cell_counts_check < min_cells_required).any():
    print(" Some participants have fewer than 500 cells!")
else:
    print("All participants have ≥ 500 cells.")

# 3. Check total number of cells
print(f"Total number of cells selected: {len(selected_obs)}")

In [ ]:
# Save
selected_barcodes = selected_obs.index.tolist()
pd.Series(selected_barcodes).to_csv("selected_barcodes.txt", index=False, header=False)

In [ ]:
adata_filtered = adata[adata.obs_names.isin(selected_barcodes)]
adata_filtered = adata_filtered.to_memory()

In [ ]:
# Convert string columns to categorical type
adata_filtered.obs = adata_filtered.obs.apply(lambda col: col.astype('category') if col.dtype == 'object' else col)
adata_filtered.var = adata_filtered.var.apply(lambda col: col.astype('category') if col.dtype == 'object' else col)

In [ ]:
# Subset to PMC region
pmc_adata = adata_filtered[adata_filtered.obs['brain_region'] == 'PMC'].copy()

In [ ]:
# Save
adata_filtered.write("/rds/general/user/tf424/home/subset_20_filtered.h5ad")